In [19]:
import pandas as pd
import mysql.connector

In [20]:
# Ask for config
print("Enter MySQL config:")
host = input("Host: ")
user = input("User: ")
password = input("Password: ")
database = input("Database: ")
table_name = input("Target Table: ")
file_path = input("Excel/CSV file path: ")


Enter MySQL config:


Host:  localhost
User:  root
Password:  1234
Database:  school
Target Table:  students
Excel/CSV file path:  students_100.xlsx


In [21]:
# Connect
conn = mysql.connector.connect(
    host=host,
    user=user,
    password=password,
    database=database)
cursor = conn.cursor()

In [22]:
# Read file
if file_path.endswith(".xlsx"):
    df = pd.read_excel(file_path, engine="openpyxl")
elif file_path.endswith(".csv"):
    df = pd.read_csv(file_path)
else:
    raise ValueError("File must be CSV or Excel")

print(f"Loaded {len(df)} rows with columns: {list(df.columns)}")


Loaded 100 rows with columns: ['ID', 'FullName', 'Email', 'Phone', 'Address', 'Birth_Date']


In [23]:
# Convert to list of tuples
records = df.values.tolist()
records[:5]

[[1,
  'Allison Hill',
  'jillrhodes@miller.com',
  201686579303.0,
  '386 Shane Harbors, Port Lindachester, KY 20880',
  '1969-04-02'],
 [2,
  'Gabrielle Davis',
  'melanie94@blair.com',
  nan,
  'Unit 6184 Box 9593, DPO AP 09617',
  '1957-09-19'],
 [3,
  'Patricia Galloway',
  'jamesshawn@martin-kelly.com',
  201026855092.0,
  '283 Steven Groves, Lake Mark, WI 07832',
  '1970-03-10'],
 [4,
  'Renee Morales',
  'robinbradley@edwards.info',
  201796233790.0,
  nan,
  '1997-04-19'],
 [5,
  'Michael Carlson',
  'dcarlson@hotmail.com',
  201295310485.0,
  '184 Rodriguez Mews, South Aaron, VA 35023',
  '1988-08-31']]

In [24]:
# Prepare insert query dynamically
cols = ", ".join(df.columns)
placeholders = ", ".join(["%s"] * len(df.columns))
update_stmt = ", ".join([f"{col}=VALUES({col})" for col in df.columns])

insert_sql = f"""
INSERT INTO {table_name} ({cols})
VALUES ({placeholders})
ON DUPLICATE KEY UPDATE {update_stmt}
"""


In [25]:
# Insert
batch_size = 500

for i in range(0, len(records), batch_size):
    batch = records[i:i+batch_size]  
    cursor.executemany(insert_sql, batch)
    conn.commit()
    print(f"Batch {i//batch_size + 1}: inserted/updated {len(batch)} rows")
    
print(f" Done. Total rows processed: {len(records)}")

Batch 1: inserted/updated 100 rows
 Done. Total rows processed: 100


In [26]:
# Verify row count in table
cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
print(" Rows in table now:", cursor.fetchone()[0])

 Rows in table now: 100


In [27]:
cursor.close()
conn.close()